## Phylogenetic Analysis

The goal of this notebook is to perform phylogenetic inference on the samples from the boat as well as other genomes sampled from around the same time as the boat outbreak. 

In [131]:
from Bio import SeqIO # Parse fasta files 
from itertools import combinations # Handle pairwise comparisons
from collections import Counter # Count instances in a list
from ete3 import Tree # Tree visualization


To make the consensus sequences, I took each `BAM` file and counted the occurences of each nucleotide at every position. If there were less than 100 reads that covered a given site (with `BQ > 25`) then I coded this site as an `N`. Then, using both replicates, I filled in missing (`N`) nucleotides with the more highly covered positions. 

This method doesn't take into account insertions and deletions. That could be a bit of a shortcoming, however, I can also include insertions and deletions if necessary. Since I'm mostly focused on SNPs in the intrahost population, I thoguht this was a reasonable oversight.

In [132]:
# Path to the aligned sequences
boat_genomes_path = "../../results/consensus/aligned_consensus.fa"
# Put the sequence records and ids into a dictionary structure
boat_genomes = {record.id : record.seq for record in SeqIO.parse(boat_genomes_path, "fasta")}

#### Edit Distance

Here is the edit distance (hamming distance) between each of the genomes. The most divergent sample is `10136`. We know that this sample probably wasn't infected by the same virus that spread on the boat. It also doesn't seem like any of the cases on the boat came from this person. 

In [133]:
distance = lambda x,y : sum(c1 != c2 for c1, c2 in zip(x, y))

edit_distance = {f"{g1}-{g2}": distance(boat_genomes[g1], boat_genomes[g2]) for g1, g2 in combinations(boat_genomes.keys(), 2)}
outlier_distance = {f"{g1}-{g2}": distance(boat_genomes[g1], boat_genomes[g2]) for g1, g2 in combinations(boat_genomes.keys(), 2) if g1 == '10136' or g2 == '10136'}

print(f"The mean pairwise distance between samples is {sum(v for v in edit_distance.values()) / len(edit_distance):.2f}.\n")
print(f"However, this includes a sample (10136) that is highly diverged from the rest with a mean edit distance of {sum(v for v in outlier_distance.values()) / len(outlier_distance):.2f} from any other sample.")



The mean pairwise distance between samples is 2.13.

However, this includes a sample (10136) that is highly diverged from the rest with a mean edit distance of 8.83 from any other sample.


#### Consensus Differences 

Here are all the differences between the 24 patients and the reference (Wuhan-1). I collapsed patients with the same consensus sequecnes into a single row. A better way to visualize this might be to add it to a phylogenetic tree of the samples on the boat as an annotation. 

Some of these sites are different from the reference, but the same in all 24 individuals (boat consensus mutations). I could either include or exclude these from the visualization. I'm not sure what's most prudent. 

In [134]:
# Reference Sequence 
reference = [base for record in SeqIO.parse("../../config/ref/SARS2.fa", "fasta") for base in record.seq]

# Save a dict of the consensus SNPs for each patient
consesus = {}

# Populate the dict
for patient, genome in boat_genomes.items():
    differences = []
    for i, bases in enumerate(zip(genome.upper(), reference)):
        if len(set(bases)) > 1: 
            differences.append((i, bases))
    consesus[patient] = differences

# Get a set of all unique SNPs 
consensus_differences = set(snp for snp_list in consesus.values() for snp in snp_list)

# Fill in the missing SNPs in the dict
for patient, snps in consesus.items(): 
    for snp in consensus_differences: 
        if snp not in snps:
            consesus[patient].append((snp[0], (snp[1][1], snp[1][1])))


# Condense the identical sequences
unique_consensus = {}

for patient, snps in consesus.items(): 
    con = ' '.join([snp[1][0] for snp in sorted(snps, key = (lambda s: s[0]))])
    if con in unique_consensus.keys():
        unique_consensus[con].append(patient)
    else: 
        unique_consensus[con] = [patient]

# Print the differences for each patient 
print(f"Position: {' '.join([str(snp[0]) for snp in sorted(consensus_differences, key = (lambda s: s[0]))])}")
print(f"Reference: \t\t\t\t{' '.join([snp[1][1] for snp in sorted(consensus_differences, key = (lambda s: s[0]))])}")

for snps, patients in unique_consensus.items(): 
    print(f"{'/'.join(patients)}: \t\t\t\t\t{snps}")


Position: 240 538 1058 1254 3036 6045 7563 8289 11082 13251 13422 14407 16911 17124 17155 17351 19523 23402 25562 27310 28375 29166 29298 29552 29867 29869
Reference: 				C C C A C T C C G C C C G C C C C A G T G C A G G C
10114: 					T C T A T T T C T C C T G C C C T G T T T C A G G A
10117/10118/10127/10130/10027/10040/10089/10091/10102/10110: 					T C T A T T T C T C C T G C C C C G T T T C A G G A
10128: 					T C T A T T T T T C C T G T C C C G T T T C A G G A
10129: 					T C T A T T T T T C C T G C C C C G T T T C A G G C
10131: 					T T T A T T T C T C C T G C C C C G T C T C A G G A
10136: 					T C T G T C C C G C C T T C T C C G T T G C A A G A
10138: 					T C T A T T T C T C C T G C C C C G T T T C T G G A
10028: 					T C T A T T T T T C C T G C C C C G T T T C A G A A
10029/10094/10106: 					T C T A T T T C T C C T G C C C C G T T T C A G A A
10039: 					T C T A T T T C T C C T G C C C C G T T T T A G A A
10042: 					T C T A T T T C T C T T G C C T C G T T T C A G G A
10088: 

#### Boat Consensus 

These are the mutations that are different from the reference, but shared between all 24 people we've analyzed on the boat. 

In [163]:
# Consensus in all samples including divergent 10136
all_boat_consensus = []
for k,v in Counter(snp for snps in consesus.values() for snp in snps).items():
    if v == 24:
        all_boat_consensus.append(k)
    
# Consensus in all samples EXCLUDING divergent 10136
true_boat_consensus = []
for k,v in Counter(snp for patient, snps in consesus.items() if patient != "10136" for snp in snps).items():
    if v == 23:
        true_boat_consensus.append(k)

print(f"Mutations found in every patient including 10136:\n\t{[''.join([alleles[0], str(pos), alleles[1]]) for pos, alleles in all_boat_consensus]}\n")
print(f"Mutations found in every patient EXCLUDING 10136:\n\t{[''.join([alleles[0], str(pos), alleles[1]]) for pos, alleles in true_boat_consensus]}\n")



Mutations found in every patient including 10136:
	['T240C', 'T1058C', 'T3036C', 'T14407C', 'G23402A', 'T25562G']

Mutations found in every patient EXCLUDING 10136:
	['T240C', 'T1058C', 'T3036C', 'T7563C', 'T11082G', 'T14407C', 'G23402A', 'T25562G', 'T28375G', 'C17155C', 'G16911G', 'G29552G', 'A1254A', 'T6045T']



#### Phylogenetic Tree

Here I make a tree from the consensus samples deep-sequenced from the boat. The tree was made using IQtree's automatic model using the commad `iqtree -s {input}`. 

In [177]:
t = Tree("../../results/consensus/aligned_consensus.fa.treefile")

t.show()